In [7]:
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

nltk.download("stopwords")

import string
import re

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import SGDClassifier

import numpy as np

from scipy import sparse
import datetime

import s3fs

# import lch_proprietary.ml_modeling as ml 

from sklearn.feature_extraction.text import TfidfVectorizer

import io

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natashamathur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
test = pd.read_csv("s3://advancedml-koch-mathur-hinkson/test.csv")

KeyboardInterrupt: 

In [ ]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

In [9]:
train = pd.read_csv("train.csv")

Split train.csv into train (80%) and validation sets (20%).

In [10]:
validation_set = train[:360000]

In [11]:
train_set = train[360000:]

To speed up process, create small samples of the train_set on which to run NB.  Ensure that samples are iid by replacing after each draw.

In [12]:
train_sample1 = train_set.sample(frac=0.05, replace=True, random_state=1)

In [13]:
train_sample1.shape

(72244, 45)

In [14]:
ls = LancasterStemmer()
ps = PorterStemmer() 

sw = set(stopwords.words('english'))
sw.add('')

def clean_text(text, stemming=None, remove_sw = True):

    t = text.replace("-", " ").split(" ")
    
    t = [w.lower() for w in t]
    
    if remove_sw == True:
        t = [w for w in t if w not in sw]
    
    if stemming == None:
        pass;
    elif stemming == "Porter":
        t = [ps.stem(w) for w in t]
    elif stemming == "Lancaster":
        t = [ls.stem(w) for w in t]
    else:
        print("Please enter a valid stemming type")
        
    t = [w.strip(string.punctuation) for w in t]


    return ' '.join(t)

In [15]:
def add_text_cleaning_cols(df):
    print(datetime.datetime.now())
    
    df['split'] = df["comment_text"].apply(lambda x: x.split(" "))
    df['cleaned_w_stopwords'] = df["comment_text"].apply(clean_text,args=(None,False),)

    print(datetime.datetime.now())
    df['cleaned_no_stem'] = df["comment_text"].apply(clean_text,)
    df['cleaned_porter'] = df["comment_text"].apply(clean_text,args=("Porter",),)
    df['cleaned_lancaster'] = df["comment_text"].apply(clean_text,args=("Lancaster",),)

    print(datetime.datetime.now())

    df['perc_upper'] = df["comment_text"].apply(lambda x: round((len(re.findall(r'[A-Z]',x)) / len(x)), 3))

    df['num_exclam'] = df["comment_text"].apply(lambda x:(len(re.findall(r'!',x))))
    
    df['num_words'] = df["split"].apply(lambda x: len(x))
    print(datetime.datetime.now())

    df['perc_stopwords'] = round((df.num_words - df['cleaned_no_stem'].apply(lambda x: len(x)))/df.num_words,3) 
    
    df['num_upper_words'] = df["split"].apply(lambda x: sum(map(str.isupper, x)) 
)
    print(datetime.datetime.now())

    
    

In [16]:
add_text_cleaning_cols(train_sample1)

2019-05-26 10:32:40.038149
2019-05-26 10:32:42.277086
2019-05-26 10:33:54.874809
2019-05-26 10:33:55.733431
2019-05-26 10:33:56.392296


In [20]:
train_sample1.to_csv('processed_train_sample.csv')

In [21]:
train_sample1.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count', 'split',
       'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster', 'perc_upper', 'num_exclam', 'num_words',
       'perc_stopwords', 'num_upper_words'],
      dtype='object')

Pickle the dataset and send to s3 bucket:

In [18]:

pickle_buffer = io.BytesIO()
s3_resource = boto3.resource('s3')
bucket='advancedml-koch-mathur-hinkson'
key='preprocessed_train_sample_50pct.pkl'

# test.to_pickle(pickle_buffer)
# s3_resource.Object(bucket, 'full_preprocessed_test.pkl').put(Body=pickle_buffer.getvalue())

test.to_pickle(key)
s3_resource.Object(bucket,key).put(Body=open(key, 'rb'))


NameError: name 'boto3' is not defined

In [22]:
addtl_cols = ['perc_upper', 'num_exclam', 'num_words', 'perc_stopwords', 'num_upper_words']
#addtl_cols = ['perc_upper']

In [23]:
train_sample1.shape

(72244, 55)

In [24]:
train_sample1.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,split,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words
488037,842255,0.0,"What, you don't think the earth is at least 8,...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[What,, you, don't, think, the, earth, is, at,...","what you don't think the earth is at least 8,0...","what think earth least 8,000 years old","what think earth least 8,000 year old","what think ear least 8,000 year old",0.017,0,12,-2.167,0
851755,5162644,0.0,"IMO one reason is the ""degree of foresight"". ...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[IMO, one, reason, is, the, ""degree, of, fores...",imo one reason is the degree of foresight tho...,imo one reason degree foresight less foresight...,imo one reason degre foresight less foresight ...,imo on reason degree foresight less foresight ...,0.019,0,53,-1.660,1
830924,5137657,0.0,""" Oh- and throw in some more ""racist"" and ""mys...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"["", Oh-, and, throw, in, some, more, ""racist"",...",oh and throw in some more racist and mysogen...,oh throw racist mysogeny name calls supporter...,oh throw racist mysogeny name call support wo...,oh throw racist mysogeny nam cal support wond...,0.013,0,36,-3.306,0
851263,5162070,0.0,If anyone understands the subtle intricacies o...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[If, anyone, understands, the, subtle, intrica...",if anyone understands the subtle intricacies o...,anyone understands subtle intricacies saying o...,anyon understand subtl intricaci say one thing...,anyon understand subtl int say on thing vot an...,0.015,0,34,-3.441,0
1196489,5578177,0.0,"""klu klux klan""?",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[""klu, klux, klan""?]",klu klux klan,klu klux klan,klu klux klan,klu klux klan,0.000,0,3,-3.333,0


Due to memory issues, we needed to use a smaller training set.  We used a random iid sample of half of the train_sample1 frame to train NB model:

In [25]:
subset_train_sample1 = train_sample1.sample(frac=0.5, replace=True, random_state=1)

In [26]:
def assign_toxicity(target):
    if target > 0.5:
        return 1
    else:
        return 0

Calculated a binary "toxic" or "non toxic" class designation based on the target score

In [27]:
subset_train_sample1['cat'] = subset_train_sample1.target.apply(assign_toxicity,)

In [28]:
toxic = subset_train_sample1[subset_train_sample1.cat == 1]
nontoxic = subset_train_sample1[subset_train_sample1.cat == 0]

In [29]:
subset_train_sample1.shape, toxic.shape, nontoxic.shape

((36122, 56), (2204, 56), (33918, 56))

Reshaping the dataset to be include an equal number of toxic and nontoxic samples

In [30]:
sample_df = toxic.append(toxic).append(nontoxic.sample(2204*2))
sample_df = sample_df.sample(frac=1).reset_index(drop=True)

print(sample_df.cat.value_counts())


1    4408
0    4408
Name: cat, dtype: int64


In [31]:
sample_df[sample_df.identity_attack > .5]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words,cat
29,5289517,0.700000,"But Flynn is white, they don't lynch white peo...",0.100000,0.000000,0.600000,0.200000,0.200000,0.0,0.0,...,but flynn is white they don't lynch white peop...,flynn white lynch white people...#maga,flynn white lynch white people...#maga,flyn white lynch whit people...#maga,0.105,0,9,-3.222,0,1
55,5002890,0.600000,"Muslims like to be run by dictators, kings, Em...",0.100000,0.000000,0.600000,0.300000,0.000000,0.0,0.0,...,muslims like to be run by dictators kings emir...,muslims like run dictators kings emir ever lik...,muslim like run dictators kings emir ever like...,muslim lik run dictators kings emir ever lik s...,0.033,0,95,-1.779,2,1
175,752699,0.600000,"Except if you were female, a person of colour,...",0.000000,0.000000,0.600000,0.000000,0.000000,0.0,0.0,...,except if you were female a person of colour o...,except female person colour gay ran right disc...,except female person colour gay ran right disc...,exceiv female person colour gay ran right disc...,0.019,0,35,-2.886,0,1
261,6317204,0.706667,How racist of you! We all know black people an...,0.013333,0.026667,0.693333,0.400000,0.013333,0.0,0.0,...,how racist of you we all know black people and...,racist you know black people black secret serv...,racist you know black peopl black secret servi...,rac you know black peopl black secret servicem...,0.056,2,20,-2.750,1,1
369,5653377,0.700000,"For them, bashing white man is leading.",0.000000,0.000000,0.600000,0.200000,0.100000,0.0,0.0,...,for them bashing white man is leading,them bashing white man leading,them bash white man leading,them bash whit man leading,0.026,0,7,-3.286,0,1
387,752699,0.600000,"Except if you were female, a person of colour,...",0.000000,0.000000,0.600000,0.000000,0.000000,0.0,0.0,...,except if you were female a person of colour o...,except female person colour gay ran right disc...,except female person colour gay ran right disc...,exceiv female person colour gay ran right disc...,0.019,0,35,-2.886,0,1
511,998923,0.802015,"Good, we have to make a buck somewhere, just k...",0.059277,0.122703,0.786603,0.634855,0.016598,NaN,NaN,...,good we have to make a buck somewhere just kee...,good make buck somewhere keep pissed africans ...,good make buck somewhere keep piss african cou...,good mak buck somewhere keep piss afr country ...,0.019,0,22,-1.636,0,1
525,6264639,1.000000,"God bless you, guys. Don't let those Islamist ...",0.000000,0.000000,1.000000,0.166667,0.000000,0.0,0.0,...,god bless you guys don't let those islamist lo...,god bless you guys let islamist loving teacher...,god bless you guys let islamist love teacher d...,god bless you guys let islam lov teach destroy...,0.037,0,13,-3.692,0,1
531,6180167,0.951578,ERIC----YOU are a MORON\nWEATHER or NOT is HOW...,0.271491,0.439064,0.910773,0.892274,0.004353,NaN,NaN,...,eric you are a moron\nweather or not is how...,eric moron\nweather nigers muslums talk \nwhat...,eric moron\nweath niger muslum talk \nwhat r u,er moron\nweather nig muslum talk \nwhat r u,0.558,0,15,-2.200,9,1
592,5778917,1.000000,"Yeah, socialists want to exterminate Jews, lik...",0.000000,0.166667,1.000000,1.000000,0.166667,0.0,0.0,...,yeah socialists want to exterminate jews like ...,yeah socialists want exterminate jews like jew...,yeah socialist want extermin jews like jewish ...,yeah soc want extermin jews lik jew guy berny ...,0.028,0,36,-3.556,0,1


Because we are unable to train an NB model on categorical (text) and continuous (numerical) data at the same time, our action plan changed to running two independent models for each type of data and then running a thrid NB model on the resulting predict_proba from the other two trained models.

In [32]:
def run_model(model_df, train_perc=.80, addtl_feats =[''], model_type = "Multi", 
              num_iter = 10, should_print=False, see_inside=False, comments="comment_text",
             target='cat'):
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df[target].values
    
    # tokenizing text
#     count_vect = CountVectorizer()
#     X_all_counts = count_vect.fit_transform(X_all.astype('U'))
    #print(X_all_counts.shape)

    # calculating frequencies
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    fitted_vectorizer=tfidf_vectorizer.fit(model_df[comments])
    X_all_tfidf =  fitted_vectorizer.transform(model_df[comments])


    print(X_all_tfidf.shape)
    
    if addtl_feats != ['']: # combine non-text and text features if necessary
        print("here")
#         others_all = model_df[addtl_feats].values.reshape(-1,1)

        others_all = model_df[addtl_feats].values.reshape(-1,len(addtl_feats))
        #print(others_all)
        newfeatures_all = sparse.hstack((X_all_tfidf, others_all.astype(float))).tocsr()
    else:
        newfeatures_all = X_all_tfidf
    
    
    X_train = newfeatures_all[train_start:train_end]
    y_train = model_df[train_start:train_end][target].values
    y_train=y_train.astype('int')
    

    X_test = newfeatures_all[test_start:test_end]
    y_test = model_df[test_start:test_end][target].values
    
    
    
    
    if model_type == 'Multi':
        clf = MultinomialNB().fit(X_train, y_train)
    if model_type == "Gauss":
        clf = GaussianNB().fit(X_train, y_train) 
        
    preds_for_train = clf.predict(X_train)
    
    
   
    predicted = clf.predict(X_test)
    accuracy = np.mean(predicted == y_test)
    
    output = model_df[test_start:test_end]
    output['predicted'] = predicted
    output['y_test'] = y_test
    output['accuracy'] = output.predicted == output.y_test
    

#     y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
#     precision = precision_score(y_true_sorted, preds)


    if should_print == True:

        print("The accuracy on the test set is {}%.".format(round(accuracy*100,2)))    
    
    if see_inside == True:
        return clf, accuracy, X_all_counts, X_all_tfidf
    else:
        return clf, accuracy, preds_for_train, predicted, output


In [33]:
clf1, accuracy, preds_for_train, predicted , output = run_model(sample_df, comments = "cleaned_lancaster", should_print=False)

print("The unique values predicted in the training set include :" + str(np.unique(preds_for_train)))
print("The unique values predicted in the test set include :" + str(np.unique(predicted)))

(8816, 17676)
The unique values predicted in the training set include :[0 1]
The unique values predicted in the test set include :[0 1]


In [34]:
output.y_test.value_counts()

1    884
0    879
Name: y_test, dtype: int64

In [35]:
output.predicted.value_counts()

1    1039
0     724
Name: predicted, dtype: int64

In [36]:
accuracy

0.8349404424276801

In [37]:
targets = output[output.y_test == 1]
targets[targets.accuracy == True].shape[0] / targets.shape[0]

0.9230769230769231

In [38]:
output[output.y_test == 0].accuracy.value_counts()

True     656
False    223
Name: accuracy, dtype: int64

In [39]:
targets[targets.identity_attack > .8].accuracy.value_counts()

True    9
Name: accuracy, dtype: int64

In [65]:
all_models = pd.DataFrame()
all_models = all_models.reindex(columns = ['model_type', 'text', 'train_perc', 'uvals_train', 'uvals_test', 'accuracy', 'target_accuracy'])
all_models

,model_type,text,train_perc,uvals_train,uvals_test,accuracy,target_accuracy


In [66]:
top_acc = 0
best_model = ''
model_characteristics = []

for tp in [.6, .7, .8]:
    for mt in ['Multi', 'Gauss']:
        for text in ['cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter','cleaned_lancaster']:
            
            instance_characteristics = [tp, mt, text]
            print(instance_characteristics)

            clf, accuracy, preds_for_train, predicted, output = run_model(sample_df, comments = text, should_print=False)

            uvals_train, uvals_test = str(np.unique(preds_for_train)), str(np.unique(predicted))
            
            print("The unique values predicted for the training set include: {}".format(uvals_train))
            print("The unique values predicted for the test set include: " + uvals_test)

            targets = output[output.y_test == 1]
            target_accuracy = targets[targets.accuracy == True].shape[0] / targets.shape[0]

            print("Accuracy: {} , Target Accuracy: {}".format(accuracy, target_accuracy))
            print()
            
            all_models = all_models.append(pd.Series([mt, text, tp, uvals_train, uvals_test, accuracy, target_accuracy], 
                                                     index=all_models.columns ), ignore_index=True)

            if target_accuracy > top_acc:
                print("CHANGED")
                print(instance_characteristics, target_accuracy)
                model_characteristics = instance_characteristics
                top_acc = target_accuracy
                model = clf
                
            print()



[0.6, 'Multi', 'cleaned_w_stopwords']
(8816, 20855)
The unique values predicted for the training set include: [0 1]
The unique values predicted for the test set include: [0 1]
Accuracy: 0.8258650028360749 , Target Accuracy: 0.9355203619909502

CHANGED
[0.6, 'Multi', 'cleaned_w_stopwords'] 0.9355203619909502

[0.6, 'Multi', 'cleaned_no_stem']
(8816, 20852)
The unique values predicted for the training set include: [0 1]
The unique values predicted for the test set include: [0 1]
Accuracy: 0.8281338627339762 , Target Accuracy: 0.9355203619909502


[0.6, 'Multi', 'cleaned_porter']
(8816, 18692)
The unique values predicted for the training set include: [0 1]
The unique values predicted for the test set include: [0 1]
Accuracy: 0.8389109472490074 , Target Accuracy: 0.9321266968325792


[0.6, 'Multi', 'cleaned_lancaster']
(8816, 17676)
The unique values predicted for the training set include: [0 1]
The unique values predicted for the test set include: [0 1]
Accuracy: 0.8349404424276801 , Targ

In [79]:
all_models[all_models.target_accuracy >= top_acc].sort_values(by='accuracy', ascending=False).head(1)

,model_type,text,train_perc,uvals_train,uvals_test,accuracy,target_accuracy
1,Multi,cleaned_no_stem,0.6,[0 1],[0 1],0.828134,0.93552
